# Multiselect map in JS
from memory i guess SO DRAMATIC lol

In [1]:
import numpy as np
import pandas as pd

import bokeh.io

import panel as pn
import datetime as dt

from bokeh.tile_providers import CARTODBPOSITRON, get_provider

tile_provider = get_provider(CARTODBPOSITRON)

bokeh.io.output_notebook()

pn.extension()

Loading BokehJS ...

# Load and prep data

In [2]:
# Helper fxn to set coords right
def mercator_creator(df, lat='lat', lon='lon'):
    # from https://www.youtube.com/watch?v=BojxegBh9_4
    
    k = 6378137
    df['x'] = k * np.radians(df[lon])
    df['y'] = np.log(np.tan((90 + df[lat]) * np.pi / 360)) * k
    
    return df

In [3]:
# Load in the metadata
ferm_locations = pd.read_csv("./data/ferm.csv")

# Split the Lat-Long column into two columns, Lat and Long
ferm_locations[['Lat', 'Long']] = ferm_locations['Lat-Long'].str.split(', ', 1, expand=True)

# Create a new dataframe only if the point has a valid Lat and Long
ferm_locations_final = ferm_locations[ferm_locations['Lat'].notna()].copy()

# Convert from a string, and put in columns called Latitude and Longitude
ferm_locations_final.loc[:,'Latitude1'] = pd.to_numeric(ferm_locations_final['Lat'],errors='coerce')
ferm_locations_final.loc[:,'Longitude1'] = pd.to_numeric(ferm_locations_final['Long'],errors='coerce')

# Add x, y for lat long
ferm_locations_final = mercator_creator(ferm_locations_final, lat='Latitude1', lon='Longitude1')

class_options = list(ferm_locations_final.Class.unique())
country_options = list(ferm_locations_final.Country_obtained.unique())

ferm_locations_final.Acquisition_date = pd.to_datetime(ferm_locations_final.Acquisition_date)
ferm_locations_final.Extract_date = pd.to_datetime(ferm_locations_final.Extract_date)
ferm_locations_final.Arrival_date = pd.to_datetime(ferm_locations_final.Arrival_date)

# Bokeh (Faster)

In [4]:
date_range_slider = pn.widgets.DatetimeRangePicker(
    name='Show Samples Between:',
    enable_time = False,
    start=dt.datetime(2021, 12, 1), end=dt.datetime(2022, 8, 1),
    value=(dt.datetime(2021, 12, 1), dt.datetime(2022, 8, 1))
)

In [5]:
country_choice = pn.widgets.MultiChoice(name='Country', value=country_options,
    options=country_options, solid=False)

In [21]:
color_map

{'Dairy': '#1f77b4',
 'Control': '#aec7e8',
 'Bread': '#ff7f0e',
 'Bean': '#ffbb78',
 'Drink': '#2ca02c',
 'Vegetable': '#98df8a',
 'Fruit': '#d62728',
 'Fish': '#ff9896',
 'Vinegar': '#9467bd',
 'Rice': '#c5b0d5',
 'Shellfish': '#8c564b',
 'Intestine': '#c49c94',
 'Meat': '#e377c2',
 'Crustacean': '#f7b6d2',
 'Unclear': '#7f7f7f'}

In [37]:
# .choices__inner
for col in color_map:
    print(f".choices__inner div[data-value={col}] {{\n\tbackground-color: {color_map[col]} !important;\n}}")

.choices__inner div[data-value=Dairy] {
	background-color: #1f77b4 !important;
}
.choices__inner div[data-value=Control] {
	background-color: #aec7e8 !important;
}
.choices__inner div[data-value=Bread] {
	background-color: #ff7f0e !important;
}
.choices__inner div[data-value=Bean] {
	background-color: #ffbb78 !important;
}
.choices__inner div[data-value=Drink] {
	background-color: #2ca02c !important;
}
.choices__inner div[data-value=Vegetable] {
	background-color: #98df8a !important;
}
.choices__inner div[data-value=Fruit] {
	background-color: #d62728 !important;
}
.choices__inner div[data-value=Fish] {
	background-color: #ff9896 !important;
}
.choices__inner div[data-value=Vinegar] {
	background-color: #9467bd !important;
}
.choices__inner div[data-value=Rice] {
	background-color: #c5b0d5 !important;
}
.choices__inner div[data-value=Shellfish] {
	background-color: #8c564b !important;
}
.choices__inner div[data-value=Intestine] {
	background-color: #c49c94 !important;
}
.choices__inner

In [6]:
class_choice = pn.widgets.MultiChoice(name='Choose Ferment Class:', value=class_options,
    options=class_options, css_classes=[])

In [7]:
colors = bokeh.palettes.Category20[15]

color_map = dict(zip(class_options, colors))

ferm_locations_final['color'] = ferm_locations_final['Class'].map(color_map)

In [8]:
ferm_locations_final['url']  = ["https://github.com/lianamerk/hovercal/blob/main/examples/fruit_hovercal_1.png?raw=true"] * len(ferm_locations_final)

In [9]:
p = bokeh.plotting.figure(
        frame_height=400,
        frame_width=700,
        x_axis_label="Latitude",
        y_axis_label="Longitude",
        tools =['wheel_zoom', 'pan', 'reset', 'tap'],
        x_axis_type = 'mercator',
        y_axis_type = 'mercator',
        x_range = [ferm_locations_final.x.min()-100000, ferm_locations_final.x.max() + 100000],
        y_range = [ferm_locations_final.y.min()-100000, ferm_locations_final.y.max() + 100000]
    )

p.toolbar_location = 'above'
p.add_tile(tile_provider)
p.add_tools(bokeh.models.BoxZoomTool(match_aspect=True))

hover = bokeh.models.HoverTool()

chosen_cds = bokeh.models.ColumnDataSource(
    {
        "x": ferm_locations_final["x"],
        "y": ferm_locations_final["y"],
        "color": ferm_locations_final['color'],
        "Country_obtained": ferm_locations_final['Country_obtained'],
        "Class": ferm_locations_final['Class'],
        "Sample_ID": ferm_locations_final['Sample_ID'],
        "Type": ferm_locations_final['Type'],
        "url": ferm_locations_final['url'],
    }
)

# hover.tooltips = """
# <font face="Arial" size="3">
# <div>
#     <div><strong>Ferment Class:  </strong>@Class</div>
#     <div><strong>ID: </strong>@Sample_ID</div>
#     <div><strong>ID: </strong>@Type</div>

# </div>
# </font>
# """

hover.tooltips = """
    <div>
        <div>
            <img
                src="@imgs" height="42" alt="@imgs" width="42"
                style="float: left; margin: 0px 15px 15px 0px;"
                border="2"
            ></img>
        </div>
        <div>
            <span style="font-size: 17px; font-weight: bold;">@Class</span>
            <span style="font-size: 15px; color: #966;">@Sample_ID</span>
        </div>
        <div>
            <span><b>bold</b></span>
        </div>
        <div>
            <span style="font-size: 15px;">Location</span>
            <span style="font-size: 10px; color: #696;">@x</span>
        </div>
    </div>
"""
p.add_tools(hover)

url = "https://github.com/lianamerk/hovercal/blob/main/examples/fruit_hovercal_1.png?raw=true"
taptool = p.select(type=bokeh.models.TapTool)
taptool.callback = bokeh.models.OpenURL(url=url)

In [10]:
# point available
cds = bokeh.models.ColumnDataSource(
    {
        "x": ferm_locations_final["x"],
        "y": ferm_locations_final["y"],
        "color": ferm_locations_final['color'],
        "Country_obtained": ferm_locations_final['Country_obtained'],
        "Class": ferm_locations_final['Class'],
        "Sample_ID": ferm_locations_final['Sample_ID'],
        "Type": ferm_locations_final['Type'],
        "url": ferm_locations_final['url'],
    }
)

In [11]:
circle = p.circle(
    source=chosen_cds, x="x", y="y", fill_color="color", line_color="color",
)

In [12]:
# JavaScript code for the callback stored as a string
jscode = """
let selected_class = class_choice.value;
let selected_country = country_choice.value;

let point_visible = source_visible.data;
let point_available = source_available.data;

point_visible.x = []
point_visible.y = []
point_visible.color = []
point_visible.Country_obtained = []
point_visible.Class = []
point_visible.Sample_ID = []
point_visible.Type = []
point_visible.url = []


for (let i = 0; i < point_available.x.length; i++) {
    if (selected_class.includes(point_available.Class[i]) && selected_country.includes(point_available.Country_obtained[i])) {
        point_visible.x.push(point_available.x[i]);
        point_visible.y.push(point_available.y[i]);
        point_visible.color.push(point_available.color[i]);
        point_visible.Country_obtained.push(point_available.Country_obtained[i]);
        point_visible.Class.push(point_available.Class[i]);
        point_visible.Sample_ID.push(point_available.Sample_ID[i]);
        point_visible.Type.push(point_available.Type[i]);
        point_visible.url.push(point_available.url[i]);
    }
}

source_visible.change.emit(); 
"""

In [13]:
class_choice.jscallback(value=jscode, args=dict(source_available=cds, source_visible=chosen_cds, class_choice=class_choice, country_choice=country_choice))


Callback(args={'source_available': ColumnDataSource(id='1055', ...), 'source_visible': ColumnDataSource(id='1043', ...), 'class_choice': MultiChoice(name='Choose Ferment Class:', options=['Dairy', 'Control', ...], value=['Dairy', 'Control', ...]), 'country_choice': MultiChoice(name='Country', options=['France', 'Singapore', ...], solid=False, value=['France', 'Singapore', ...])}, code={'value': '\nlet selected_class = class_choice.value;\nlet selected_country = country_choice.value;\n\nlet point_visible = source_visible.data;\nlet point_available = source_available.data;\n\npoint_visible.x = []\npoint_visible.y = []\npoint_visible.color = []\npoint_visible.Country_obtained = []\npoint_visible.Class = []\npoint_visible.Sample_ID = []\npoint_visible.Type = []\npoint_visible.url = []\n\n\nfor (let i = 0; i < point_available.x.length; i++) {\n    if (selected_class.includes(point_available.Class[i]) && selected_country.includes(point_available.Country_obtained[i])) {\n        point_visible

In [14]:
country_choice.jscallback(value=jscode, args=dict(source_available=cds, source_visible=chosen_cds, class_choice=class_choice, country_choice=country_choice))

Callback(args={'source_available': ColumnDataSource(id='1055', ...), 'source_visible': ColumnDataSource(id='1043', ...), 'class_choice': MultiChoice(name='Choose Ferment Class:', options=['Dairy', 'Control', ...], value=['Dairy', 'Control', ...]), 'country_choice': MultiChoice(name='Country', options=['France', 'Singapore', ...], solid=False, value=['France', 'Singapore', ...])}, code={'value': '\nlet selected_class = class_choice.value;\nlet selected_country = country_choice.value;\n\nlet point_visible = source_visible.data;\nlet point_available = source_available.data;\n\npoint_visible.x = []\npoint_visible.y = []\npoint_visible.color = []\npoint_visible.Country_obtained = []\npoint_visible.Class = []\npoint_visible.Sample_ID = []\npoint_visible.Type = []\npoint_visible.url = []\n\n\nfor (let i = 0; i < point_available.x.length; i++) {\n    if (selected_class.includes(point_available.Class[i]) && selected_country.includes(point_available.Country_obtained[i])) {\n        point_visible

In [15]:
p_pane = pn.pane.Bokeh(p)

In [16]:
map_panel = pn.Row(p, pn.Spacer(width=15), pn.Column(class_choice, country_choice))
map_panel

Row
    [0] Bokeh(Figure)
    [1] Spacer(width=15)
    [2] Column
        [0] MultiChoice(name='Choose Ferment Class:', options=['Dairy', 'Control', ...], value=['Dairy', 'Control', ...])
        [1] MultiChoice(name='Country', options=['France', 'Singapore', ...], solid=False, value=['France', 'Singapore', ...])

In [17]:
map_panel.save('panelJS.html', embed=True)
